<a href="https://colab.research.google.com/github/naga-dheeraj-p/Assignments/blob/main/Scalping_bnfty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# fut_scalping

In [ ]:
df=pd.read_csv("/content/gdrive/MyDrive/bankfut/final_BANKNIFTY_F2.csv")

In [ ]:
import pandas as pd

# Step 0: Drop rows where Date is NaN
df = df.dropna(subset=['Date'])

# Step 1: Convert float Date to int, then to string
df['Date'] = df['Date'].astype(int).astype(str)

# Step 2: Convert Date to datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Step 3: Format Date as dd-mm-yyyy
df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')

# Step 4: Combine Date and Time into one column
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d-%m-%Y %H:%M')

# Step 5: Sort by Datetime
df = df.sort_values(by='Datetime').reset_index(drop=True)

# Optional: Move 'Datetime' to front
df = df[['Datetime'] + [col for col in df.columns if col != 'Datetime']]

df


,Datetime,Symbol,Date,Time,Open,High,Low,Close,Volume,Open Interest
0,2018-01-01 09:16:00,BANKNIFTY_F2,01-01-2018,09:16,25634.0,25634.0,25608.0,25608.0,80.0,NaN
1,2018-01-01 09:16:00,BANKNIFTY_F2,01-01-2018,09:16,25634.0,25634.0,25608.0,25608.0,80.0,NaN
2,2018-01-01 09:18:00,BANKNIFTY_F2,01-01-2018,09:18,25643.1,25643.1,25643.1,25643.1,80.0,NaN
3,2018-01-01 09:18:00,BANKNIFTY_F2,01-01-2018,09:18,25643.1,25643.1,25643.1,25643.1,80.0,NaN
4,2018-01-01 09:20:00,BANKNIFTY_F2,01-01-2018,09:20,25631.5,25631.5,25631.5,25631.5,40.0,NaN
...,...,...,...,...,...,...,...,...,...,...
912432,2023-02-28 15:29:00,BANKNIFTY_F2,28-02-2023,15:29,40710.0,40710.0,40701.0,40706.0,250.0,1173225.0
912433,2023-02-28 15:30:00,BANKNIFTY_F2,28-02-2023,15:30,40706.0,40710.0,40705.0,40710.0,550.0,1173225.0
912434,2023-02-28 15:30:00,BANKNIFTY_F2,28-02-2023,15:30,40706.0,40710.0,40705.0,40710.0,550.0,1173225.0
912435,2023-02-28 18:45:00,BANKNIFTY_F2,28-02-2023,18:45,40706.7,40706.7,40706.7,40706.7,1.0,1172925.0


In [ ]:
import pandas as pd

# 1. Sort the DataFrame by Datetime
df = df.sort_values(by='Datetime').reset_index(drop=True)

# 2. Initialize trade tracking lists
entries = []
exits = []
pnl = []

position = None
entry_price = 0
stop_loss = 0
target_price = 0

for i in range(len(df)-1):  # up to second-last candle
    if position is None:
        # BUY condition
        if df.loc[i, 'Close'] - df.loc[i, 'Open'] > 100:
            position = 'long'
            entry_price = df.loc[i+1, 'Open']
            stop_loss = df.loc[i, 'Open']
            target_price = entry_price + 150
            entries.append((df.loc[i+1, 'Datetime'], 'BUY', entry_price))
        # SELL condition
        elif df.loc[i, 'Open'] - df.loc[i, 'Close'] > 100:
            position = 'short'
            entry_price = df.loc[i+1, 'Open']
            stop_loss = df.loc[i, 'Open']
            target_price = entry_price - 150
            entries.append((df.loc[i+1, 'Datetime'], 'SELL', entry_price))
    else:
        # Monitor open/high/low/close of current candle
        price = df.loc[i, 'Close']

        if position == 'long':
            # Trailing stoploss for long: adjust stop loss if price increases
            stop_loss = max(stop_loss, price - 20)

            if price <= stop_loss or price >= target_price:
                pnl.append(price - entry_price)
                exits.append((df.loc[i, 'Datetime'], 'EXIT', price))
                position = None

        elif position == 'short':
            # Trailing stoploss for short: adjust stop loss downward
            stop_loss = min(stop_loss, price + 20)

            if price >= stop_loss or price <= target_price:
                pnl.append(entry_price - price)
                exits.append((df.loc[i, 'Datetime'], 'EXIT', price))
                position = None

# 3. Convert results to DataFrames for analysis
entry_df = pd.DataFrame(entries, columns=['Datetime', 'Action', 'Price'])
exit_df = pd.DataFrame(exits, columns=['Datetime', 'Action', 'Price'])
pnl_df = pd.DataFrame({'PnL': pnl})

# 4. Summary
print("Total Trades:", len(pnl_df))
print("Total PnL:", pnl_df['PnL'].sum())
print("Average PnL per Trade:", pnl_df['PnL'].mean())


Total Trades: 1400
Total PnL: 183874.19999999995
Average PnL per Trade: 131.33871428571425


2018

In [ ]:
import pandas as pd

# Ensure Datetime column is datetime type
df['Datetime'] = pd.to_datetime(df['Datetime'])

# Filter only 2018 data
df_2018 = df[(df['Datetime'] >= '2018-01-01') & (df['Datetime'] < '2019-01-01')]

# Sort by Datetime
df_2018 = df_2018.sort_values(by='Datetime').reset_index(drop=True)

# Initialize trade tracking lists
entries = []
exits = []
pnl = []

position = None
entry_price = 0
stop_loss = 0
target_price = 0

for i in range(len(df_2018)-1):  # up to second-last candle
    if position is None:
        # BUY condition
        if df_2018.loc[i, 'Close'] - df_2018.loc[i, 'Open'] > 100:
            position = 'long'
            entry_price = df_2018.loc[i+1, 'Open']
            stop_loss = df_2018.loc[i, 'Open']
            target_price = entry_price + 150
            entries.append((df_2018.loc[i+1, 'Datetime'], 'BUY', entry_price))
        # SELL condition
        elif df_2018.loc[i, 'Open'] - df_2018.loc[i, 'Close'] > 100:
            position = 'short'
            entry_price = df_2018.loc[i+1, 'Open']
            stop_loss = df_2018.loc[i, 'Open']
            target_price = entry_price - 150
            entries.append((df_2018.loc[i+1, 'Datetime'], 'SELL', entry_price))
    else:
        # Monitor close price for stoploss/target
        price = df_2018.loc[i, 'Close']

        if position == 'long':
            # Trailing stoploss for long: adjust stop loss if price increases
            stop_loss = max(stop_loss, price - 20)

            if price <= stop_loss or price >= target_price:
                pnl.append(price - entry_price)
                exits.append((df_2018.loc[i, 'Datetime'], 'EXIT', price))
                position = None

        elif position == 'short':
            # Trailing stoploss for short: adjust stop loss downward
            stop_loss = min(stop_loss, price + 20)

            if price >= stop_loss or price <= target_price:
                pnl.append(entry_price - price)
                exits.append((df_2018.loc[i, 'Datetime'], 'EXIT', price))
                position = None

# Convert results to DataFrames
entry_df = pd.DataFrame(entries, columns=['Datetime', 'Action', 'Price'])
exit_df = pd.DataFrame(exits, columns=['Datetime', 'Action', 'Price'])
pnl_df = pd.DataFrame({'PnL': pnl})

# Summary
print("Total Trades:", len(pnl_df))
print("Total PnL:", pnl_df['PnL'].sum())
print("Average PnL per Trade:", pnl_df['PnL'].mean())


Total Trades: 40
Total PnL: 5280.749999999993
Average PnL per Trade: 132.0187499999998


In [ ]:
import pandas as pd

# After your existing code where you generate entries, exits, pnl

# We'll create a combined trade log from entries, exits and pnl

# First, sanity check length of all lists (should be equal)
assert len(entries) == len(exits) == len(pnl), "Mismatch in trade records lengths"

trade_log = []

for i in range(len(pnl)):
    entry_dt, action, entry_price = entries[i]
    exit_dt, _, exit_price = exits[i]
    profit_loss_points = pnl[i]
    profit_loss_pct = (profit_loss_points / entry_price) * 100

    # Determine if exit was due to target or stoploss
    # We know targets and stoplosses from your logic:
    # For longs: target_price = entry_price + 150
    # stop_loss dynamically adjusted trailing stoploss
    # For shorts: target_price = entry_price - 150
    # stop_loss adjusted similarly

    # Since stop_loss changes during trade, we'll approximate exit reason:
    # If abs(profit_loss_points) >= 150 → "Target Hit"
    # Else "Stoploss Hit"
    if action == 'BUY':
        if profit_loss_points >= 150:
            exit_reason = 'Target Hit'
        else:
            exit_reason = 'Stoploss Hit'
    else:  # SELL
        if profit_loss_points >= 150:
            exit_reason = 'Target Hit'
        else:
            exit_reason = 'Stoploss Hit'

    trade_log.append({
        'Entry Datetime': entry_dt,
        'Entry Price': entry_price,
        'Action': action,
        'Exit Datetime': exit_dt,
        'Exit Price': exit_price,
        'Points Captured': profit_loss_points,
        'Profit/Loss %': profit_loss_pct,
        'Exit Reason': exit_reason
    })

trade_log_df = pd.DataFrame(trade_log)

# Show first 10 trades as sample
trade_log_df


,Entry Datetime,Entry Price,Action,Exit Datetime,Exit Price,Points Captured,Profit/Loss %,Exit Reason
0,2018-01-18 09:16:00,26820.00,SELL,2018-01-18 09:16:00,26589.60,230.40,0.859060,Target Hit
1,2018-01-25 09:16:00,27637.30,SELL,2018-01-25 09:16:00,27430.10,207.20,0.749711,Target Hit
2,2018-01-31 09:16:00,27247.40,BUY,2018-01-31 09:18:00,27400.00,152.60,0.560053,Target Hit
3,2018-02-06 14:02:00,25675.00,BUY,2018-02-06 14:04:00,25785.00,110.00,0.428432,Stoploss Hit
4,2018-02-06 14:24:00,26020.00,BUY,2018-02-06 14:25:00,26183.50,163.50,0.628363,Target Hit
5,2018-02-09 09:16:00,25749.90,SELL,2018-02-09 09:17:00,25582.10,167.80,0.651653,Target Hit
6,2018-02-09 09:33:00,25546.60,BUY,2018-02-09 09:34:00,25600.00,53.40,0.209030,Stoploss Hit
7,2018-04-03 09:16:00,24355.20,BUY,2018-04-03 09:20:00,24520.00,164.80,0.676652,Target Hit
8,2018-04-09 09:16:00,24885.30,BUY,2018-04-09 09:18:00,25000.00,114.70,0.460915,Stoploss Hit
9,2018-04-23 09:16:00,25185.10,SELL,2018-04-23 09:16:00,24950.70,234.40,0.930709,Target Hit


In [ ]:
import pandas as pd

# Assuming df is already loaded
df['Datetime'] = pd.to_datetime(df['Datetime'])
df_2018 = df[(df['Datetime'] >= '2018-01-01') & (df['Datetime'] < '2019-01-01')]
df_2018 = df_2018.sort_values(by='Datetime').reset_index(drop=True)

entries = []
exits = []
pnl = []

position = None
entry_price = 0
stop_loss = 0
target_price = 0

i = 0
while i < len(df_2018) - 2:
    if position is None:
        candle = df_2018.loc[i]
        next_candle = df_2018.loc[i+1]

        if candle['Close'] - candle['Open'] > 60:
            position = 'long'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']  # Fixed stoploss
            target_price = entry_price + 100
            entries.append((next_candle['Datetime'], 'BUY', entry_price))
            i += 2
        elif candle['Open'] - candle['Close'] > 60:
            position = 'short'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']  # Fixed stoploss
            target_price = entry_price - 100
            entries.append((next_candle['Datetime'], 'SELL', entry_price))
            i += 2
        else:
            i += 1
    else:
        candle = df_2018.loc[i]
        price = candle['Close']

        if position == 'long':
            if price <= stop_loss or price >= target_price:
                pnl_trade = price - entry_price
                pnl.append(pnl_trade)
                exits.append((candle['Datetime'], 'EXIT', price))
                position = None
            i += 1

        elif position == 'short':
            if price >= stop_loss or price <= target_price:
                pnl_trade = entry_price - price
                pnl.append(pnl_trade)
                exits.append((candle['Datetime'], 'EXIT', price))
                position = None
            i += 1

# Trade log
trade_log = []
for idx in range(len(pnl)):
    entry_dt, action, entry_price = entries[idx]
    exit_dt, _, exit_price = exits[idx]
    points = pnl[idx]
    profit_pct = round((points / entry_price) * 100, 4)
    exit_reason = 'Target Hit' if abs(points) >= 150 else 'Stoploss Hit'

    trade_log.append({
        'Entry Datetime': entry_dt,
        'Entry Price': entry_price,
        'Action': action,
        'Exit Datetime': exit_dt,
        'Exit Price': exit_price,
        'Points Captured': points,
        'Profit/Loss %': profit_pct,
        'Exit Reason': exit_reason
    })

trade_log_df = pd.DataFrame(trade_log)

total_pct_return = trade_log_df['Profit/Loss %'].sum()
num_trades = len(trade_log_df)

print("\nTotal % Return:", round(total_pct_return, 2))
print("Total Number of Trades:", num_trades)

# Return bucket
bins = [-float('inf'), -1, 0, 1, 2, 3, float('inf')]
labels = ['<= -1%', '-1% to 0%', '0% to 1%', '1% to 2%', '2% to 3%', '> 3%']
trade_log_df['Return Bucket'] = pd.cut(trade_log_df['Profit/Loss %'], bins=bins, labels=labels)

bucket_counts = trade_log_df['Return Bucket'].value_counts().sort_index()
print("\nReturn Distribution:")
print(bucket_counts)
trade_log_df


Total % Return: 47.73
Total Number of Trades: 169

Return Distribution:
Return Bucket
<= -1%         1
-1% to 0%     50
0% to 1%     117
1% to 2%       1
2% to 3%       0
> 3%           0
Name: count, dtype: int64


,Entry Datetime,Entry Price,Action,Exit Datetime,Exit Price,Points Captured,Profit/Loss %,Exit Reason,Return Bucket
0,2018-01-12 09:16:00,25708.80,BUY,2018-01-12 12:34:00,25705.00,-3.80,-0.0148,Stoploss Hit,-1% to 0%
1,2018-01-15 09:16:00,25858.60,BUY,2018-01-15 10:03:00,25958.70,100.10,0.3871,Stoploss Hit,0% to 1%
2,2018-01-17 09:16:00,25902.60,BUY,2018-01-17 09:18:00,26004.30,101.70,0.3926,Stoploss Hit,0% to 1%
3,2018-01-18 09:16:00,26820.00,SELL,2018-01-18 09:17:00,26640.90,179.10,0.6678,Target Hit,0% to 1%
4,2018-01-18 14:50:00,26517.30,SELL,2018-01-18 14:57:00,26517.70,-0.40,-0.0015,Stoploss Hit,-1% to 0%
...,...,...,...,...,...,...,...,...,...
164,2018-12-06 09:16:00,26654.90,SELL,2018-12-06 09:17:00,26529.50,125.40,0.4705,Stoploss Hit,0% to 1%
165,2018-12-10 09:16:00,26324.95,BUY,2018-12-10 09:20:00,26428.00,103.05,0.3915,Stoploss Hit,0% to 1%
166,2018-12-11 09:16:00,25826.90,BUY,2018-12-11 09:17:00,25969.05,142.15,0.5504,Stoploss Hit,0% to 1%
167,2018-12-20 09:16:00,27358.85,SELL,2018-12-20 09:20:00,27362.35,-3.50,-0.0128,Stoploss Hit,-1% to 0%


In [ ]:
import pandas as pd

# Assuming df is already loaded
df['Datetime'] = pd.to_datetime(df['Datetime'])
df_2018 = df[(df['Datetime'] >= '2018-01-01') & (df['Datetime'] < '2019-01-01')]
df_2018 = df_2018.sort_values(by='Datetime').reset_index(drop=True)

entries = []
exits = []
pnl = []

position = None
entry_price = 0
stop_loss = 0
target_price = 0

i = 0
while i < len(df_2018) - 2:
    if position is None:
        candle = df_2018.loc[i]
        next_candle = df_2018.loc[i+1]

        if candle['Close'] - candle['Open'] > 60:
            position = 'long'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']
            target_price = entry_price + 100
            entries.append((next_candle['Datetime'], 'BUY', entry_price))
            i += 2
        elif candle['Open'] - candle['Close'] > 60:
            position = 'short'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']
            target_price = entry_price - 100
            entries.append((next_candle['Datetime'], 'SELL', entry_price))
            i += 2
        else:
            i += 1
    else:
        while i < len(df_2018):
            candle = df_2018.loc[i]

            if position == 'long':
                if candle['Low'] <= stop_loss:
                    pnl_trade = stop_loss - entry_price
                    pnl.append(pnl_trade)
                    exits.append((candle['Datetime'], 'EXIT', stop_loss))
                    position = None
                    i += 1
                    break
                elif candle['High'] >= target_price:
                    pnl_trade = target_price - entry_price
                    pnl.append(pnl_trade)
                    exits.append((candle['Datetime'], 'EXIT', target_price))
                    position = None
                    i += 1
                    break
                else:
                    i += 1

            elif position == 'short':
                if candle['High'] >= stop_loss:
                    pnl_trade = entry_price - stop_loss
                    pnl.append(pnl_trade)
                    exits.append((candle['Datetime'], 'EXIT', stop_loss))
                    position = None
                    i += 1
                    break
                elif candle['Low'] <= target_price:
                    pnl_trade = entry_price - target_price
                    pnl.append(pnl_trade)
                    exits.append((candle['Datetime'], 'EXIT', target_price))
                    position = None
                    i += 1
                    break
                else:
                    i += 1

# Construct trade log
trade_log = []
for idx in range(len(pnl)):
    entry_dt, action, entry_price = entries[idx]
    exit_dt, _, exit_price = exits[idx]
    points = pnl[idx]
    profit_pct = round((points / entry_price) * 100, 4)
    exit_reason = 'Target Hit' if abs(points) >= 100 else 'Stoploss Hit'

    trade_log.append({
        'Entry Datetime': entry_dt,
        'Entry Price': entry_price,
        'Action': action,
        'Exit Datetime': exit_dt,
        'Exit Price': exit_price,
        'Points Captured': f"{'+' if points >= 0 else ''}{points}",
        'Profit/Loss %': f"{'+' if profit_pct >= 0 else ''}{profit_pct}",
        'Exit Reason': exit_reason
    })

trade_log_df = pd.DataFrame(trade_log)

# Summary
total_pct_return = sum(float(p.strip('%')) for p in trade_log_df['Profit/Loss %'])
num_trades = len(trade_log_df)

print("\nTotal % Return:", round(total_pct_return, 2))
print("Total Number of Trades:", num_trades)

# Return buckets
trade_log_df['Profit/Loss % Numeric'] = trade_log_df['Profit/Loss %'].astype(str).str.replace('%', '').astype(float)

bins = [-float('inf'), -1, 0, 1, 2, 3, float('inf')]
labels = ['<= -1%', '-1% to 0%', '0% to 1%', '1% to 2%', '2% to 3%', '> 3%']
trade_log_df['Return Bucket'] = pd.cut(trade_log_df['Profit/Loss % Numeric'], bins=bins, labels=labels)

bucket_counts = trade_log_df['Return Bucket'].value_counts().sort_index()
print("\nReturn Distribution:")
print(bucket_counts)

# Final trade log output
trade_log_df.drop(columns=['Profit/Loss % Numeric'], inplace=True)
trade_log_df



Total % Return: 44.84
Total Number of Trades: 176

Return Distribution:
Return Bucket
<= -1%         0
-1% to 0%     46
0% to 1%     130
1% to 2%       0
2% to 3%       0
> 3%           0
Name: count, dtype: int64


,Entry Datetime,Entry Price,Action,Exit Datetime,Exit Price,Points Captured,Profit/Loss %,Exit Reason,Return Bucket
0,2018-01-12 09:16:00,25708.80,BUY,2018-01-12 10:04:00,25808.80,+100.0,+0.389,Target Hit,0% to 1%
1,2018-01-15 09:16:00,25858.60,BUY,2018-01-15 10:02:00,25958.60,+100.0,+0.3867,Target Hit,0% to 1%
2,2018-01-17 09:16:00,25902.60,BUY,2018-01-17 09:18:00,26002.60,+100.0,+0.3861,Target Hit,0% to 1%
3,2018-01-18 09:16:00,26820.00,SELL,2018-01-18 09:17:00,26720.00,+100.0,+0.3729,Target Hit,0% to 1%
4,2018-01-18 14:50:00,26517.30,SELL,2018-01-18 14:57:00,26517.30,+0.0,+0.0,Stoploss Hit,-1% to 0%
...,...,...,...,...,...,...,...,...,...
171,2018-12-06 09:16:00,26654.90,SELL,2018-12-06 09:17:00,26554.90,+100.0,+0.3752,Target Hit,0% to 1%
172,2018-12-10 09:16:00,26324.95,BUY,2018-12-10 09:17:00,26424.95,+100.0,+0.3799,Target Hit,0% to 1%
173,2018-12-11 09:16:00,25826.90,BUY,2018-12-11 09:17:00,25926.90,+100.0,+0.3872,Target Hit,0% to 1%
174,2018-12-20 09:16:00,27358.85,SELL,2018-12-20 09:20:00,27358.85,+0.0,+0.0,Stoploss Hit,-1% to 0%


In [ ]:
import pandas as pd

# Ensure your dataframe is already loaded as df with columns: ['Datetime', 'Open', 'High', 'Low', 'Close']
df['Datetime'] = pd.to_datetime(df['Datetime'])
df_2018 = df[(df['Datetime'] >= '2018-01-01') & (df['Datetime'] < '2019-01-01')]
df_2018 = df_2018.sort_values(by='Datetime').reset_index(drop=True)

entries = []
exits = []
pnl = []

position = None
entry_price = 0
stop_loss = 0
target_price = 0

i = 0
while i < len(df_2018) - 2:
    if position is None:
        candle = df_2018.loc[i]
        next_candle = df_2018.loc[i + 1]

        if candle['Close'] - candle['Open'] > 60:
            position = 'long'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']
            target_price = entry_price + 100
            entries.append((next_candle['Datetime'], 'BUY', entry_price))
            i += 2
        elif candle['Open'] - candle['Close'] > 60:
            position = 'short'
            entry_price = next_candle['Open']
            stop_loss = candle['Open']
            target_price = entry_price - 100
            entries.append((next_candle['Datetime'], 'SELL', entry_price))
            i += 2
        else:
            i += 1
    else:
        while i < len(df_2018):
            candle = df_2018.loc[i]
            dt = candle['Datetime']

            if position == 'long':
                if candle['Low'] <= stop_loss:
                    pnl_trade = stop_loss - entry_price
                    pnl.append(pnl_trade)
                    exits.append((dt, 'EXIT', stop_loss))
                    print(f"LONG STOPLOSS HIT | Entry: {entry_price}, Exit: {stop_loss}, PnL: {pnl_trade}")
                    position = None
                    i += 1
                    break
                elif candle['High'] >= target_price:
                    pnl_trade = target_price - entry_price
                    pnl.append(pnl_trade)
                    exits.append((dt, 'EXIT', target_price))
                    print(f"LONG TARGET HIT | Entry: {entry_price}, Exit: {target_price}, PnL: {pnl_trade}")
                    position = None
                    i += 1
                    break

            elif position == 'short':
                if candle['High'] >= stop_loss:
                    pnl_trade = entry_price - stop_loss
                    pnl.append(pnl_trade)
                    exits.append((dt, 'EXIT', stop_loss))
                    print(f"SHORT STOPLOSS HIT | Entry: {entry_price}, Exit: {stop_loss}, PnL: {pnl_trade}")
                    position = None
                    i += 1
                    break
                elif candle['Low'] <= target_price:
                    pnl_trade = entry_price - target_price
                    pnl.append(pnl_trade)
                    exits.append((dt, 'EXIT', target_price))
                    print(f"SHORT TARGET HIT | Entry: {entry_price}, Exit: {target_price}, PnL: {pnl_trade}")
                    position = None
                    i += 1
                    break

            i += 1

# Construct trade log
trade_log = []
for idx in range(len(pnl)):
    entry_dt, action, entry_price = entries[idx]
    exit_dt, _, exit_price = exits[idx]
    points = pnl[idx]
    profit_pct = round((points / entry_price) * 100, 4)
    exit_reason = 'Target Hit' if abs(points) >= 100 else 'Stoploss Hit'

    trade_log.append({
        'Entry Datetime': entry_dt,
        'Entry Price': entry_price,
        'Action': action,
        'Exit Datetime': exit_dt,
        'Exit Price': exit_price,
        'Points Captured': f"{'+' if points >= 0 else ''}{points}",
        'Profit/Loss %': f"{'+' if profit_pct >= 0 else ''}{profit_pct}",
        'Exit Reason': exit_reason
    })

trade_log_df = pd.DataFrame(trade_log)

# Summary
total_pct_return = sum([float(p.strip('+')) for p in trade_log_df['Profit/Loss %']])
num_trades = len(trade_log_df)

print("\nTotal % Return:", round(total_pct_return, 2))
print("Total Number of Trades:", num_trades)

# Return bucket
trade_log_df['Profit/Loss % (num)'] = trade_log_df['Profit/Loss %'].astype(str).str.replace('+', '').astype(float)
bins = [-float('inf'), -1, 0, 1, 2, 3, float('inf')]
labels = ['<= -1%', '-1% to 0%', '0% to 1%', '1% to 2%', '2% to 3%', '> 3%']
trade_log_df['Return Bucket'] = pd.cut(trade_log_df['Profit/Loss % (num)'], bins=bins, labels=labels)

bucket_counts = trade_log_df['Return Bucket'].value_counts().sort_index()
print("\nReturn Distribution:")
print(bucket_counts)

# Final output
trade_log_df.drop(columns=['Profit/Loss % (num)'], inplace=True)
trade_log_df


LONG TARGET HIT | Entry: 25708.8, Exit: 25808.8, PnL: 100.0
LONG TARGET HIT | Entry: 25858.6, Exit: 25958.6, PnL: 100.0
LONG TARGET HIT | Entry: 25902.6, Exit: 26002.6, PnL: 100.0
SHORT TARGET HIT | Entry: 26820.0, Exit: 26720.0, PnL: 100.0
SHORT STOPLOSS HIT | Entry: 26517.3, Exit: 26517.3, PnL: 0.0
SHORT TARGET HIT | Entry: 27637.3, Exit: 27537.3, PnL: 100.0
LONG STOPLOSS HIT | Entry: 27365.1, Exit: 27365.1, PnL: 0.0
LONG TARGET HIT | Entry: 27396.6, Exit: 27496.6, PnL: 100.0
LONG TARGET HIT | Entry: 27247.4, Exit: 27347.4, PnL: 100.0
SHORT TARGET HIT | Entry: 27600.0, Exit: 27500.0, PnL: 100.0
SHORT TARGET HIT | Entry: 27500.0, Exit: 27400.0, PnL: 100.0
SHORT TARGET HIT | Entry: 27422.7, Exit: 27322.7, PnL: 100.0
SHORT STOPLOSS HIT | Entry: 27250.0, Exit: 27340.0, PnL: -90.0
LONG TARGET HIT | Entry: 27282.8, Exit: 27382.8, PnL: 100.0
SHORT STOPLOSS HIT | Entry: 27411.7, Exit: 27411.7, PnL: 0.0
LONG STOPLOSS HIT | Entry: 27514.7, Exit: 27514.7, PnL: 0.0
LONG TARGET HIT | Entry: 27545

,Entry Datetime,Entry Price,Action,Exit Datetime,Exit Price,Points Captured,Profit/Loss %,Exit Reason,Return Bucket
0,2018-01-12 09:16:00,25708.80,BUY,2018-01-12 10:04:00,25808.80,+100.0,+0.389,Target Hit,0% to 1%
1,2018-01-15 09:16:00,25858.60,BUY,2018-01-15 10:02:00,25958.60,+100.0,+0.3867,Target Hit,0% to 1%
2,2018-01-17 09:16:00,25902.60,BUY,2018-01-17 09:18:00,26002.60,+100.0,+0.3861,Target Hit,0% to 1%
3,2018-01-18 09:16:00,26820.00,SELL,2018-01-18 09:17:00,26720.00,+100.0,+0.3729,Target Hit,0% to 1%
4,2018-01-18 14:50:00,26517.30,SELL,2018-01-18 14:57:00,26517.30,+0.0,+0.0,Stoploss Hit,-1% to 0%
...,...,...,...,...,...,...,...,...,...
171,2018-12-06 09:16:00,26654.90,SELL,2018-12-06 09:17:00,26554.90,+100.0,+0.3752,Target Hit,0% to 1%
172,2018-12-10 09:16:00,26324.95,BUY,2018-12-10 09:17:00,26424.95,+100.0,+0.3799,Target Hit,0% to 1%
173,2018-12-11 09:16:00,25826.90,BUY,2018-12-11 09:17:00,25926.90,+100.0,+0.3872,Target Hit,0% to 1%
174,2018-12-20 09:16:00,27358.85,SELL,2018-12-20 09:20:00,27358.85,+0.0,+0.0,Stoploss Hit,-1% to 0%


In [ ]:
trade_log_df.head(20)

,Entry Datetime,Entry Price,Action,Exit Datetime,Exit Price,Points Captured,Profit/Loss %,Exit Reason,Return Bucket
0,2018-01-12 09:16:00,25708.8,BUY,2018-01-12 10:04:00,25808.8,+100.0,+0.389,Target Hit,0% to 1%
1,2018-01-15 09:16:00,25858.6,BUY,2018-01-15 10:02:00,25958.6,+100.0,+0.3867,Target Hit,0% to 1%
2,2018-01-17 09:16:00,25902.6,BUY,2018-01-17 09:18:00,26002.6,+100.0,+0.3861,Target Hit,0% to 1%
3,2018-01-18 09:16:00,26820.0,SELL,2018-01-18 09:17:00,26720.0,+100.0,+0.3729,Target Hit,0% to 1%
4,2018-01-18 14:50:00,26517.3,SELL,2018-01-18 14:57:00,26517.3,+0.0,+0.0,Stoploss Hit,-1% to 0%
5,2018-01-25 09:16:00,27637.3,SELL,2018-01-25 09:17:00,27537.3,+100.0,+0.3618,Target Hit,0% to 1%
6,2018-01-25 13:58:00,27365.1,BUY,2018-01-25 14:00:00,27365.1,+0.0,+0.0,Stoploss Hit,-1% to 0%
7,2018-01-29 09:16:00,27396.6,BUY,2018-01-29 09:17:00,27496.6,+100.0,+0.365,Target Hit,0% to 1%
8,2018-01-31 09:16:00,27247.4,BUY,2018-01-31 09:17:00,27347.4,+100.0,+0.367,Target Hit,0% to 1%
9,2018-02-01 11:41:00,27600.0,SELL,2018-02-01 12:13:00,27500.0,+100.0,+0.3623,Target Hit,0% to 1%


In [ ]:
import pandas as pd

# Ensure df is defined beforehand:
# df = pd.read_csv(...)  or however you import the data
df['Datetime'] = pd.to_datetime(df['Datetime'])

for year in range(2018, 2023):
    print(f"\n{'='*30}\nYear: {year}\n{'='*30}")

    df_year = df[(df['Datetime'] >= f'{year}-01-01') & (df['Datetime'] < f'{year+1}-01-01')]
    df_year = df_year.sort_values(by='Datetime').reset_index(drop=True)

    entries = []
    exits = []
    pnl = []

    position = None
    entry_price = 0
    stop_loss = 0
    target_price = 0

    i = 0
    while i < len(df_year) - 2:
        if position is None:
            candle = df_year.loc[i]
            next_candle = df_year.loc[i + 1]

            if candle['Close'] - candle['Open'] > 100:
                position = 'long'
                entry_price = next_candle['Open']
                stop_loss = candle['Open']
                target_price = entry_price + 100
                entries.append((next_candle['Datetime'], 'BUY', entry_price))
                i += 2
            elif candle['Open'] - candle['Close'] > 100:
                position = 'short'
                entry_price = next_candle['Open']
                stop_loss = candle['Open']
                target_price = entry_price - 100
                entries.append((next_candle['Datetime'], 'SELL', entry_price))
                i += 2
            else:
                i += 1
        else:
            while i < len(df_year):
                candle = df_year.loc[i]
                dt = candle['Datetime']

                if position == 'long':
                    if candle['Low'] <= stop_loss:
                        pnl_trade = stop_loss - entry_price
                        pnl.append(pnl_trade)
                        exits.append((dt, 'EXIT', stop_loss))
                        position = None
                        i += 1
                        break
                    elif candle['High'] >= target_price:
                        pnl_trade = target_price - entry_price
                        pnl.append(pnl_trade)
                        exits.append((dt, 'EXIT', target_price))
                        position = None
                        i += 1
                        break

                elif position == 'short':
                    if candle['High'] >= stop_loss:
                        pnl_trade = entry_price - stop_loss
                        pnl.append(pnl_trade)
                        exits.append((dt, 'EXIT', stop_loss))
                        position = None
                        i += 1
                        break
                    elif candle['Low'] <= target_price:
                        pnl_trade = entry_price - target_price
                        pnl.append(pnl_trade)
                        exits.append((dt, 'EXIT', target_price))
                        position = None
                        i += 1
                        break

                i += 1

    # Construct trade log
    trade_log = []
    for idx in range(len(pnl)):
        entry_dt, action, entry_price = entries[idx]
        exit_dt, _, exit_price = exits[idx]
        points = pnl[idx]
        profit_pct = round((points / entry_price) * 100, 4)
        exit_reason = 'Target Hit' if abs(points) >= 100 else 'Stoploss Hit'

        trade_log.append({
            'Entry Datetime': entry_dt,
            'Entry Price': entry_price,
            'Action': action,
            'Exit Datetime': exit_dt,
            'Exit Price': exit_price,
            'Points Captured': f"{'+' if points >= 0 else ''}{points}",
            'Profit/Loss %': f"{'+' if profit_pct >= 0 else ''}{profit_pct}",
            'Exit Reason': exit_reason
        })

    trade_log_df = pd.DataFrame(trade_log)

    # Summary
    total_pct_return = sum([float(p.strip('+')) for p in trade_log_df['Profit/Loss %']])
    num_trades = len(trade_log_df)

    print("Total % Return:", round(total_pct_return, 2))
    print("Total Number of Trades:", num_trades)

    # Return bucket
    trade_log_df['Profit/Loss % (num)'] = trade_log_df['Profit/Loss %'].astype(str).str.replace('+', '', regex=False).astype(float)
    bins = [-float('inf'), -1, 0, 1, 2, 3, float('inf')]
    labels = ['<= -1%', '-1% to 0%', '0% to 1%', '1% to 2%', '2% to 3%', '> 3%']
    trade_log_df['Return Bucket'] = pd.cut(trade_log_df['Profit/Loss % (num)'], bins=bins, labels=labels)

    bucket_counts = trade_log_df['Return Bucket'].value_counts().sort_index()
    print("\nReturn Distribution:")
    print(bucket_counts)



Year: 2018
Total % Return: 14.67
Total Number of Trades: 41

Return Distribution:
Return Bucket
<= -1%        0
-1% to 0%     2
0% to 1%     39
1% to 2%      0
2% to 3%      0
> 3%          0
Name: count, dtype: int64

Year: 2019
Total % Return: 13.58
Total Number of Trades: 139

Return Distribution:
Return Bucket
<= -1%        4
-1% to 0%    55
0% to 1%     80
1% to 2%      0
2% to 3%      0
> 3%          0
Name: count, dtype: int64

Year: 2020
Total % Return: 174.86
Total Number of Trades: 601

Return Distribution:
Return Bucket
<= -1%        12
-1% to 0%     81
0% to 1%     508
1% to 2%       0
2% to 3%       0
> 3%           0
Name: count, dtype: int64

Year: 2021
Total % Return: 77.57
Total Number of Trades: 317

Return Distribution:
Return Bucket
<= -1%         0
-1% to 0%     27
0% to 1%     290
1% to 2%       0
2% to 3%       0
> 3%           0
Name: count, dtype: int64

Year: 2022
Total % Return: 48.79
Total Number of Trades: 209

Return Distribution:
Return Bucket
<= -1%    

# Options scalping